Dit tijdelijke document verkent de mogelijke targetvariabelen om de gestelde vragen vanuit ProRail te beantwoorden.

ProRail is opzoek naar voorspelling wat betreft functieherstel aan het begin van een storing, door de gegeven dataset van storingen in het verleden.

De geleverde informatie over mogelijke featurevariabelen die volgens de business-expert hoogstwaarschijnlijk grote invloed op de uiteindelijke duur van functieherstel van het spoor zijn, is hoogstwaarschijnlijk pas bij het arriveren van de monteur beschikbaar. Er vanuit gaande dat er dan pas een voorspelling gemaakt kan worden (in ieder geval met het eerste model), moet verkend worden of te tijd van monteur-aankomst tot functieherstel beschikbaar is in de dataset.

Dit document is in afwachting van de conclusie over targetvariabelen vanuit de business-interviews. Om productief te zijn als team, is parralel werken op het moment redelijk essentieel. Omdat ik hier nog op wacht, rust dit document wel op een aantal aannames over de informatie die daar uit zal komen.

Eerst laden we wat libraries, en de dataset in:

In [ ]:
import numpy as np
import pandas as pd

dataset = pd.read_csv("sap_storing_data_hu_project.csv")

Van de business-interviews weten we dat het te-voorspellen variabele het tijdstip van 'herstel' is. In de dataset staat een kolom voor tijdstip van 'functieherstel' en een tijdstip van 'einde storing'. Laten we deze in makkelijkere variabelenamen opslaan, en inspecteren:

In [ ]:
fh = "stm_fh_ddt"
dataset[fh].head(5)

In [ ]:
es = "stm_sap_storeind_ddt"
dataset[es].head(5)

In eerste oogopslag lijken de waardes in deze tabel gelijk te lijken. Laten we kijken voor welk deel van de dataset dat geld.

In [ ]:
def percentage(prob: float) -> float:
    """Convert a probability (p > 0 ∧ p < 1) to a percentage.
    Convenience function.

    Args:
        prob: probability between 0 and 1.

    Returns:
        percentage."""
    return prob * 100


n_rows = dataset.shape[0]
n_equal = (dataset[fh] == dataset[es]).sum()

f"{percentage(n_equal / n_rows)}%"

We krijgen hier 16% uit, wat niet enorm overeen lijkt te komen met de resultaten die we handmatig kunnen observeren. Hoe zit het met de datatypes?

In [ ]:
dataset[[fh, es]].dtypes

Objects, laten we dat aanpassen.

In [ ]:
dataset[[fh, es]] = dataset[[fh, es]].convert_dtypes()
dataset[[fh, es]].dtypes

Nu zijn strings. Eigenlijk willen we datetimes, dus laten we kijken of dat aangepast kan. Waarschijnlijk worden er een aantal dan niet meegenomen, dus even controleren hoeveel nans er zijn.

In [ ]:
f"{dataset[fh].isna().sum()} and {dataset[es].isna().sum()} out of {dataset.shape[0]}"

Even kijken of pandas het een beetje automatisch can converten

In [ ]:
new_fh = pd.to_datetime(dataset[fh], errors='coerce')
new_se = pd.to_datetime(dataset[es], errors='coerce')

f"{new_fh.isna().sum()} and {new_se.isna().sum()} out of {dataset.shape[0]}"

Goed genoeg.
Nu we deze omgezet hebben, kunnen we kijken of deze gelijk zijn aan elkaar.

In [ ]:
n_equal = (new_fh == new_se).sum()

f"{percentage(n_equal / n_rows)}%"

Nog steeds niet heel erg.

Waarschijnlijk willen we van een van deze 2 aftrekken wanneer de monteur aankwam.

Dit is waarschijnlijk opgemaakt uit: stm_aanntpl_tijd en stm_aanntpl_dd.

Of mss stm_aanntpl_ddt, daar staat wel een vraagteken bij in de dataset.

In [ ]:
dataset[["stm_aanntpl_tijd", "stm_aanntpl_dd", "stm_aanntpl_ddt"]]

In [ ]:
for col in dataset[["stm_aanntpl_tijd", "stm_aanntpl_dd", "stm_aanntpl_ddt"]].columns:
    print(dataset[col].isna().sum())

 Er is een kolom, stm_progfh_in_duur, en een kolom stm_fh_duur. Is een van deze 2 gelijk aan wat we net uitgerekend hebben?

In [ ]:
dataset[["stm_progfh_in_duur", "stm_fh_duur"]].head(5)

Die eerste is waarschijnlijk niet heel nuttig. Die 2e dan? Dat zal dan waarschijnlijk in minuten, of uren zijn. Allebei bekijken.

In [ ]:
tgt1 = "stm_fh_duur"
dataset[tgt1].dtypes

In [ ]:
dataset[tgt1].isna().sum()

In [ ]:
dataset[tgt1].head(5)

Wss is dat het targetvariabele. 